# SAITS Imputation with PyPOTS

> Notebook adapted in parts from [PyPOTS](https://github.com/WenjieDu/PyPOTS/).  <center>  
  <img src="https://raw.githubusercontent.com/WenjieDu/PyPOTS/main/docs/figs/PyPOTS%20logo.svg?sanitize=true" width="150" height="150" />
  </center>

<details>
        <summary> Using your own dataset </summary>
<div>   
```
{
# Install PyPOTS first: pip install pypots  
from pypots.data import mcar, fill_nan_with_mask  
from pypots.imputation import SAITS  
from pypots.utils.metrics import cal_mae  

# ❗️👀 Preprocess your data here to generate X, which should be a NumPy array
# X.shape should be [n_samples, T, D]

# hold out some observed values as ground truth
X_intact, X, missing_mask, indicating_mask = mcar(X, 0.1)
X = fill_nan_with_mask(X, missing_mask)

saits_base = SAITS(n_layers=2, d_model=256, d_inner=128, n_head=4, d_k=64, d_v=64, dropout=0.1, epochs=100)
saits_base.fit(X) # here use the whole set. You can also split X into train/val/test sets.
imputation = saits_base.impute(X)
mae = cal_mae(imputation, X_intact, indicating_mask)  # calculate mean absolute error on imputation
}```
</div>
</details>


```
# Install PyPOTS first: pip install pypots  
from pypots.data import mcar, fill_nan_with_mask  
from pypots.imputation import SAITS  
from pypots.utils.metrics import cal_mae  

# ❗️👀 Preprocess your data here to generate X, which should be a NumPy array
# X.shape should be [n_samples, T, D]

# hold out some observed values as ground truth
X_intact, X, missing_mask, indicating_mask = mcar(X, 0.1)
X = fill_nan_with_mask(X, missing_mask)

saits_base = SAITS(n_layers=2, d_model=256, d_inner=128, n_head=4, d_k=64, d_v=64, dropout=0.1, epochs=100)
saits_base.fit(X) # here use the whole set. You can also split X into train/val/test sets.
imputation = saits_base.impute(X)
mae = cal_mae(imputation, X_intact, indicating_mask)  # calculate mean absolute error on imputation
```

In [2]:
# Install PyPOTS first: pip install pypots
import numpy as np
from sklearn.preprocessing import StandardScaler

from pypots.data import load_specific_dataset, mcar, masked_fill
from pypots.imputation import SAITS, BRITS
from pypots.utils.metrics import cal_mae

# PhysionNET Dataset
## Data loading and preprocessing

In [19]:
# Data preprocessing. Tedious, but PyPOTS can help. 🤓
data = load_specific_dataset('physionet_2012')  # For datasets in PyPOTS database, PyPOTS will automatically download and extract it.
X = data['X']
num_samples = len(X['RecordID'].unique())

print('num samples: ', num_samples)
print(X)
X = X.drop('RecordID', axis = 1)
X = StandardScaler().fit_transform(X.to_numpy())
X = X.reshape(num_samples, 48, -1)
X_intact, X, missing_mask, indicating_mask = mcar(X, 0.1) # hold out 10% observed values as ground truth
X = masked_fill(X, 1 - missing_mask, np.nan)


Loading the dataset physionet_2012 with TSDB (https://github.com/WenjieDu/Time_Series_Database)...
Starting preprocessing physionet_2012...
Dataset physionet_2012 has already been downloaded. Processing directly...
Dataset physionet_2012 has already been cached. Loading from cache directly...
Loaded successfully!


/home2/joshua.wendland/Documents/sepsis/imputation/PyPOTS/pypots/data/load_specific_datasets.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_temp = df_temp.append(missing_part, ignore_index=False, sort=False)  # pad
/home2/joshua.wendland/Documents/sepsis/imputation/PyPOTS/pypots/data/load_specific_datasets.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_temp = df_temp.append(missing_part, ignore_index=False, sort=False)  # pad
/home2/joshua.wendland/Documents/sepsis/imputation/PyPOTS/pypots/data/load_specific_datasets.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_temp = df_temp.append(missing_part, ignore_index=False, sort=False)  # pad
/home2/joshua.wendland/Documents/sepsis/imputation/PyPOTS/pyp

num samples:  11988
        RecordID  ALP  ALT  AST  Albumin  BUN  Bilirubin  Cholesterol  \
0         132539  NaN  NaN  NaN      NaN  NaN        NaN          NaN   
1         132539  NaN  NaN  NaN      NaN  NaN        NaN          NaN   
2         132539  NaN  NaN  NaN      NaN  NaN        NaN          NaN   
3         132539  NaN  NaN  NaN      NaN  NaN        NaN          NaN   
4         132539  NaN  NaN  NaN      NaN  NaN        NaN          NaN   
...          ...  ...  ...  ...      ...  ...        ...          ...   
575419    163037  NaN  NaN  NaN      NaN  NaN        NaN          NaN   
575420    163037  NaN  NaN  NaN      NaN  NaN        NaN          NaN   
575421    163037  NaN  NaN  NaN      NaN  NaN        NaN          NaN   
575422    163037  NaN  NaN  NaN      NaN  NaN        NaN          NaN   
575423    163037  NaN  NaN  NaN      NaN  NaN        NaN          NaN   

        Creatinine  DiasABP  ...  RespRate  SaO2  SysABP   Temp  TroponinI  \
0              NaN      N

## Train SAITS model

In [20]:
# Model training. This is PyPOTS showtime. 💪
saits = SAITS(n_steps=48, n_features=37, n_layers=2, d_model=256, d_inner=128, n_head=4, d_k=64, d_v=64, dropout=0.1, epochs=10)
saits.fit(X)  # train the model. Here I use the whole dataset as the training set, because ground truth is not visible to the model.
imputation = saits.impute(X)  # impute the originally-missing values and artificially-missing values
mae = cal_mae(imputation, X_intact, indicating_mask)  # calculate mean absolute error on the ground truth (artificially-missing values)

No given device, using default device: cuda:0
Model initialized successfully. Number of the trainable parameters: 1378358
epoch 0: training loss 0.6657
epoch 1: training loss 0.4800
epoch 2: training loss 0.4247
epoch 3: training loss 0.3880
epoch 4: training loss 0.3613
epoch 5: training loss 0.3475
epoch 6: training loss 0.3384
epoch 7: training loss 0.3318
epoch 8: training loss 0.3272
epoch 9: training loss 0.3217
Finished training.


In [14]:
print(data.keys())
data['X'].head


dict_keys(['X', 'y'])


<bound method NDFrame.head of         RecordID  ALP  ALT  AST  Albumin  BUN  Bilirubin  Cholesterol  \
0         132539  NaN  NaN  NaN      NaN  NaN        NaN          NaN   
1         132539  NaN  NaN  NaN      NaN  NaN        NaN          NaN   
2         132539  NaN  NaN  NaN      NaN  NaN        NaN          NaN   
3         132539  NaN  NaN  NaN      NaN  NaN        NaN          NaN   
4         132539  NaN  NaN  NaN      NaN  NaN        NaN          NaN   
...          ...  ...  ...  ...      ...  ...        ...          ...   
575419    163037  NaN  NaN  NaN      NaN  NaN        NaN          NaN   
575420    163037  NaN  NaN  NaN      NaN  NaN        NaN          NaN   
575421    163037  NaN  NaN  NaN      NaN  NaN        NaN          NaN   
575422    163037  NaN  NaN  NaN      NaN  NaN        NaN          NaN   
575423    163037  NaN  NaN  NaN      NaN  NaN        NaN          NaN   

        Creatinine  DiasABP  ...  RespRate  SaO2  SysABP   Temp  TroponinI  \
0              

# Toy dataset

The data set here was generated by using a script from the **AFA** repository from *Henrik v. Kleist*.
## Imports

In [16]:
import pandas as pd
import gzip

## Load data and preprocess

In [32]:
path_toydataset = '/home2/joshua.wendland/Documents/sepsis/toy_dataset/synthetic_ts_1/synthetic_ts_test_data_eav.csv.gz'

df = pd.read_csv(path_toydataset, compression=None)
df = df.sort_values(by=['id', 'time'], ascending=True, ignore_index=True)  # time was not sorted
#df = df.loc[df['id'] == 'id_90']
num_samples = len(df['id'].unique())
print('num samples: ', num_samples)

X = df.drop('id', axis = 1)
X = df.drop(['id', 'time'], axis = 1)
X = StandardScaler().fit_transform(X.to_numpy())
# X[:,0] = df['time'].to_numpy() # time should not be normalized
X = X.reshape(num_samples, 50, -1)
X_intact, X, missing_mask, indicating_mask = mcar(X, 0.1) # hold out 10% observed values as ground truth
X = masked_fill(X, 1 - missing_mask, np.nan)

num samples:  10


In [33]:
print(X.shape)

(10, 50, 5)


# Model Training
### SAITS

In [35]:
# Model training. This is PyPOTS showtime. 💪
saits = SAITS(n_steps=50, n_features=5, n_layers=2, d_model=256, d_inner=128, n_head=4, d_k=64, d_v=64, dropout=0.0, epochs=200, patience=30)
saits.save_logs_to_tensorboard(saving_path='./runs/saits/', title='test')
saits.fit(X)  # train the model. Here I use the whole dataset as the training set, because ground truth is not visible to the model.
imputation = saits.impute(X)  # impute the originally-missing values and artificially-missing values
mae = cal_mae(imputation, X_intact, indicating_mask)  # calculate mean absolute error on the ground truth (artificially-missing values)

No given device, using default device: cuda:0
Model initialized successfully. Number of the trainable parameters: 1324864
epoch 0: training loss 1.6963
epoch 1: training loss 2.1050
epoch 2: training loss 1.6903
epoch 3: training loss 1.5936
epoch 4: training loss 1.5613
epoch 5: training loss 1.4952
epoch 6: training loss 1.4684
epoch 7: training loss 1.3944
epoch 8: training loss 1.3988
epoch 9: training loss 1.3469
epoch 10: training loss 1.3201
epoch 11: training loss 1.2794
epoch 12: training loss 1.2635
epoch 13: training loss 1.1959
epoch 14: training loss 1.2395
epoch 15: training loss 1.2541
epoch 16: training loss 1.1951
epoch 17: training loss 1.2374
epoch 18: training loss 1.2258
epoch 19: training loss 1.2286
epoch 20: training loss 1.2170
epoch 21: training loss 1.1623
epoch 22: training loss 1.1660
epoch 23: training loss 1.1601
epoch 24: training loss 1.2001
epoch 25: training loss 1.1596
epoch 26: training loss 1.1335
epoch 27: training loss 1.1593
epoch 28: training l

In [31]:
saits.logger
import tensorboard


        n_steps,
        n_features,
        rnn_hidden_size,
        learning_rate=1e-3,
        epochs=100,
        patience=10,
        batch_size=32,
        weight_decay=1e-5,
        device=None,

### BRITS



In [29]:
brits = BRITS(n_steps=50, n_features=6, rnn_hidden_size=64, learning_rate=10e-3, epochs=1000, patience=10)
brits.save_logs_to_tensorboard(saving_path='./runs/brits/', title='test')
brits.fit(X)  # train the model. Here I use the whole dataset as the training set, because ground truth is not visible to the model.
# imputation = brits.impute(X)  # impute the originally-missing values and artificially-missing values
# mae = cal_mae(imputation, X_intact, indicating_mask)  # calculate mean absolute error on the ground truth (artificially-missing values)


No given device, using default device: cuda:0
Model initialized successfully. Number of the trainable parameters: 41936
epoch 0: training loss 11.1776
epoch 1: training loss 10.9682
epoch 2: training loss 10.7608
epoch 3: training loss 10.5500
epoch 4: training loss 10.3391
epoch 5: training loss 10.1236
epoch 6: training loss 9.9127
epoch 7: training loss 9.7042
epoch 8: training loss 9.5074
epoch 9: training loss 9.3191
epoch 10: training loss 9.1319
epoch 11: training loss 8.9550
epoch 12: training loss 8.7873
epoch 13: training loss 8.6238
epoch 14: training loss 8.4655
epoch 15: training loss 8.3141
epoch 16: training loss 8.1671
epoch 17: training loss 8.0265
epoch 18: training loss 7.8895
epoch 19: training loss 7.7559
epoch 20: training loss 7.6280
epoch 21: training loss 7.5078
epoch 22: training loss 7.3886
epoch 23: training loss 7.2758
epoch 24: training loss 7.1706
epoch 25: training loss 7.0697
epoch 26: training loss 6.9794
epoch 27: training loss 6.8964
epoch 28: traini